# Prédiction des clients prospects 

In [336]:
### Import
import sys
import csv
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, recall_score, classification_report, confusion_matrix, f1_score, roc_auc_score
from sklearn import preprocessing, neighbors
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors.nearest_centroid import NearestCentroid

In [332]:
### Load data
#df_initial = pd.read_csv('df_new_initial.csv', sep=';')
df_initial = pd.read_csv('result.csv', sep=',')
del df_initial['Unnamed: 0']
df_initial

,contactid,annee_mois,campagne,campaign,civilite,code_postal,code_postal_naissance,country_connexion_name,csp,date_naissance,...,patrimoine,pays,pays_naissance,regime_matrimonial,revenus_annuels,service,timestamp,id_dim_personne,encours,cible
0,6546762003,NaN,-1.0,Source URL non trouvée,NaN,NaN,59430.0,France,15.0,NaN,...,-1.0,NaN,FRA,2.0,2.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-09-21T01:28:13.601+0200,6546762003,5.00,0
1,6742433330,NaN,456.0,REC_Liens textes autopromotion 456,0,31130.0,38000.0,France,2.0,11/04/1969,...,2.0,FRA,FRA,4.0,2.0,SFOL_AJOUTER_CB,2016-10-31T18:16:38.719+0100,6742433330,0.00,0
2,6856828837,201611.0,868.0,PAR_Onlin_Site _ECard,MR,92000.0,92150.0,France,2.0,11/05/1957,...,2.0,FRA,FRA,2.0,4.0,monprofil.PROSPECT/CREER_FICHE_PROSPECT,2016-11-21T12:01:44.736+0100,6856828837,8899.93,1
3,6754882186,NaN,300.0,Intb_Banque_Autre,1,94800.0,93420.0,France,2.0,24/06/1988,...,0.0,FRA,FRA,2.0,2.0,SFOL_AJOUTER_CB,2016-11-04T10:46:58.447+0100,6754882186,10.00,0
4,6289817192,201611.0,300.0,Intb_Banque_Autre,MR,26170.0,84600.0,Switzerland,2.0,01/05/1994,...,0.0,FRA,FRA,0.0,2.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-11-21T17:36:53.976+0100,6289817192,173.53,0
5,6748699778,NaN,307.0,REC_Onlin_MC_Googl,0,73150.0,92290.0,France,2.0,28/10/1962,...,3.0,FRA,FRA,2.0,2.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-11-01T13:44:10.556+0100,6748699778,10.00,0
6,6856855432,201612.0,307.0,REC_Onlin_MC_Googl,MR,75015.0,75015.0,France,18.0,08/10/1992,...,0.0,FRA,FRA,0.0,1.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-12-13T19:17:18.159+0100,6856855432,234.08,0
7,6821231675,201611.0,868.0,PAR_Onlin_Site _ECard,2,69001.0,69008.0,France,3.0,07/08/1989,...,0.0,FRA,FRA,0.0,1.0,SFOL_AJOUTER_CB,2016-11-15T16:46:02.913+0100,6821231675,3442.00,1
8,6747566365,NaN,868.0,PAR_Onlin_Site _ECard,2,75015.0,42000.0,France,2.0,09/06/1988,...,0.0,FRA,FRA,0.0,2.0,SFOL_AJOUTER_CB,2016-11-01T10:11:34.523+0100,6747566365,380.00,0
9,6888675097,201611.0,1543.0,REC_Onlin_Parte_Bours_SP,MLE,59161.0,59300.0,France,12.0,13/06/1996,...,0.0,FRA,FRA,0.0,0.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-11-26T17:13:18.006+0100,6888675097,319.63,0


In [333]:
#df_initial[['encours','cible']]
#df_initial.ix[44542,]
del df_initial['cible']

In [334]:
#Fixer encours des bons clients 
df_initial['cible'] = df_initial['encours'].map(lambda x: 0 if x < 1200.00 else 1).astype(int)
df_initial[['encours','cible']]

,encours,cible
0,5.00,0
1,0.00,0
2,8899.93,1
3,10.00,0
4,173.53,0
5,10.00,0
6,234.08,0
7,3442.00,1
8,380.00,0
9,319.63,0


In [335]:
df_initial[['encours','cible']]
#df = df_initial[df_initial.encours == 1100.0]
df = df_initial[df_initial.encours == 1100.0]
df
#df.ix[21339,]

,contactid,annee_mois,campagne,campaign,civilite,code_postal,code_postal_naissance,country_connexion_name,csp,date_naissance,...,patrimoine,pays,pays_naissance,regime_matrimonial,revenus_annuels,service,timestamp,id_dim_personne,encours,cible
614,6890018395,201611.0,1543.0,REC_Onlin_Parte_Bours_SP,MR,78000.0,99.0,France,2.0,18/04/1985,...,0.0,FRA,ROU,2.0,2.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-11-27T10:46:58.925+0100,6890018395,1100.0,0
8030,6861100708,201611.0,868.0,PAR_Onlin_Site _ECard,MR,75013.0,69009.0,France,16.0,22/05/1990,...,0.0,FRA,FRA,0.0,2.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-11-21T21:51:08.521+0100,6861100708,1100.0,0
10816,6856954571,201611.0,NaN,NaN,NaN,NaN,94700.0,NaN,2.0,NaN,...,0.0,NaN,FRA,6.0,2.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-11-26T21:06:10.226+0100,6856954571,1100.0,0
11468,6772337214,NaN,307.0,REC_Onlin_MC_Googl,0,67120.0,9100.0,France,18.0,28/09/1954,...,5.0,FRA,FRA,2.0,2.0,SFOL_AJOUTER_CB,2016-11-05T15:22:04.163+0100,6772337214,1100.0,0
18305,6860797763,201611.0,868.0,PAR_Onlin_Site _ECard,MLE,94440.0,99.0,France,3.0,02/12/1962,...,1.0,FRA,PRT,0.0,2.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-11-21T15:28:18.608+0100,6860797763,1100.0,0
21907,6867246484,201611.0,868.0,PAR_Onlin_Site _ECard,MR,92330.0,99.0,France,4.0,24/06/1979,...,0.0,FRA,TUN,2.0,2.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-11-22T20:19:23.050+0100,6867246484,1100.0,0
24409,6805402546,NaN,304.0,REC_Online_Affiliation Zanox,0,31000.0,31000.0,France,14.0,25/07/1985,...,1.0,FRA,FRA,0.0,4.0,SFOL_AJOUTER_CB,2016-11-11T17:04:38.191+0100,6805402546,1100.0,0
29106,6860595356,201611.0,300.0,Intb_Banque_Autre,MR,94370.0,75013.0,France,4.0,11/08/1993,...,0.0,FRA,FRA,0.0,1.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-11-21T17:52:19.134+0100,6860595356,1100.0,0
38814,7317289621,201702.0,300.0,Intb_Banque_Autre,MR,13013.0,13003.0,France,4.0,13/11/1975,...,0.0,FRA,FRA,4.0,1.0,monprofil.PROSPECT/CREER_FICHE_PROSPECT,2017-02-18T14:52:24.539+0100,7317289621,1100.0,0
39389,7312996929,201702.0,868.0,PAR_Onlin_Site _ECard,MLE,31120.0,31000.0,United Kingdom,4.0,16/08/1991,...,0.0,FRA,FRA,0.0,1.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2017-02-17T17:03:02.470+0100,7312996929,1100.0,0


In [290]:
df_initial.head()

,contactid,annee_mois,campagne,campaign,civilite,code_postal,code_postal_naissance,country_connexion_name,csp,date_naissance,...,patrimoine,pays,pays_naissance,regime_matrimonial,revenus_annuels,service,timestamp,id_dim_personne,encours,cible
0,6546762003,NaN,-1.0,Source URL non trouvée,NaN,NaN,59430.0,France,15.0,NaN,...,-1.0,NaN,FRA,2.0,2.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-09-21T01:28:13.601+0200,6546762003,5.00,0
1,6742433330,NaN,456.0,REC_Liens textes autopromotion 456,0,31130.0,38000.0,France,2.0,11/04/1969,...,2.0,FRA,FRA,4.0,2.0,SFOL_AJOUTER_CB,2016-10-31T18:16:38.719+0100,6742433330,0.00,0
2,6856828837,201611.0,868.0,PAR_Onlin_Site _ECard,MR,92000.0,92150.0,France,2.0,11/05/1957,...,2.0,FRA,FRA,2.0,4.0,monprofil.PROSPECT/CREER_FICHE_PROSPECT,2016-11-21T12:01:44.736+0100,6856828837,8899.93,1
3,6754882186,NaN,300.0,Intb_Banque_Autre,1,94800.0,93420.0,France,2.0,24/06/1988,...,0.0,FRA,FRA,2.0,2.0,SFOL_AJOUTER_CB,2016-11-04T10:46:58.447+0100,6754882186,10.00,0
4,6289817192,201611.0,300.0,Intb_Banque_Autre,MR,26170.0,84600.0,Switzerland,2.0,01/05/1994,...,0.0,FRA,FRA,0.0,2.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-11-21T17:36:53.976+0100,6289817192,173.53,0


# DF sans parrain

In [291]:
### Data management
def data_management(df):
    
    ### Fill empty value
    
    df['code_postal'] = df['code_postal'].replace('',-2)
    df['code_postal'] = pd.to_numeric(df['code_postal'])
    
    df['code_postal_naissance'] = pd.to_numeric(df['code_postal_naissance'])
   
    df['country_connexion_name'] = df['country_connexion_name'].replace(['',None],-2)
    
    df['date_naissance'] = df['date_naissance'].replace('','01/01/1800')
    
    df['flag_epargne'] = pd.to_numeric(df['flag_epargne'])
    
    #df['mailing_accord'] = df['mailing_accord'].replace(['',None],-2)
    #df['mailing_accord'] = pd.to_numeric(df['mailing_accord'])
    del df['mailing_accord']
    
    df['nature_cb'] = df['nature_cb'].replace(['',None],-2)
    df['nature_cb'] = pd.to_numeric(df['nature_cb'])
    
    df['pays_naissance'] = df['pays_naissance'].replace(['',None],-2)
    
    ### get age
    get_age(df)

    ### Get flag parrain from namesponsor
    df['Parrain'] = df['namesponsor'].map(lambda x: 0 if pd.isnull(x) else 1).astype(int)
    del df['firstnamesponsor'] 
    del df['namesponsor'] 
    
    ### Get domain from mail
    df['mail'] = df.mail.replace ([np.nan], '')
    df['domain'] = df['mail'].map(lambda x: x.split("@")[1] if x else None)
    df['domain'] = df.domain.str.lower()
    del df['mail']

    ### Get Sex from Civility
    df['Sex'] = df['civilite'].map(get_sex)
    del df['civilite']
    
    ### Delete useless cols
    del df['annee_mois']
    del df['id_dim_temps']
    del df['id_dim_personne']
    del df['encours']
    del df['timestamp']
    del df['service']
    del df['campaign']
    del df['contactid']
    del df['pays']
    
    ### transform to numeric when possible
    df = df.apply(lambda x: pd.to_numeric(x, errors='ignore'))
    df['country_connexion_name'] = df['country_connexion_name'].replace ([np.nan], '')
    df['domain'] = df['domain'].replace ([np.nan], '')
    df['pays_naissance'] = df['pays_naissance'].replace ([np.nan], '')
    df.loc[df['country_connexion_name'].value_counts()[df['country_connexion_name']].values < 40, 'country_connexion_name'] = 'other'
    df.loc[df['domain'].value_counts()[df['domain']].values < 40, 'domain'] = 'other'
    df.loc[df['pays_naissance'].value_counts()[df['pays_naissance']].values < 40, 'pays_naissance'] = 'other'
    
    
    ### Get country connexion name
    df['country_connexion'] = df['country_connexion_name'].map(process_country_connexion)
    del df['country_connexion_name']
    ### Get pays de naissance
    df['pays_de_naissance'] = df['pays_naissance'].map(process_pays_naissance)
    del df['pays_naissance']
    
    ### Process the domain
    df = process_domain (df)
    
    ### Drop NaN 
    df = df.dropna()
    
    features_df = df.drop('cible', axis=1)
    target = df['cible']
    return features_df, target, df


def get_sex(x):
    if x == "0":
        return 1
    elif (x == "1"):
        return 0
    elif (x == "2"):
        return 0
    elif (x == "MR"):
        return 1
    elif (x == "MLE"):
        return 0
    elif (x == "MME"):
        return 0
    else :
        return -1

def get_age(X):
    from datetime import datetime
    now = datetime.now()
    X['AGE'] = X['date_naissance'].dropna()
    X['AGE'] = pd.to_datetime(X['AGE'],errors='coerce')
    X['AGE'] = X['AGE'].map(lambda x : now.year-x.year  if now.month-x.month>0 else now.year - x.year -1 )
    del X['date_naissance']
    
def process_country_connexion(x):
    
    if x == 'France':
        return 1
    elif (x == 'United Kingdom'):
        return 2
    elif (x == 'other'):
        return 3
    elif (x == 'Germany'):
        return 4
    elif (x == 'Switzerland'):
        return 5
    elif (x == 'Netherlands'):
        return 6
    elif (x == 'Europe'):
        return 7
    elif (x == 'United States'):
        return 8
    elif (x == 'Reunion'):
        return 9
    elif (x==-2):
        return -2
    
def process_pays_naissance(x):

    if x == 'FRA':
        return 1
    elif (x == 'ITA'):
        return 2
    elif (x == 'other'):
        return 3
    elif (x == 'DZA'):
        return 4
    elif (x == 'BEN'):
        return 5
    elif (x == 'ESP'):
        return 6
    elif (x == 'VNM'):
        return 7
    elif (x == 'DEU'):
        return 8
    elif (x == 'MAR'):
        return 9
    elif (x=='CIV'):
        return 10
    elif (x=='CMR'):
        return 10
    elif (x=='GTO'):
        return 11
    elif (x=='SEN'):
        return 12
    elif (x=='BEL'):
        return 13
    elif (x=='CHN'):
        return 14
    elif (x=='ROU'):
        return 15
    elif (x=='BRA'):
        return 16
    elif (x=='MDG'):
        return 17
    elif (x=='PRT'):
        return 18
    elif (x=='GBR'):
        return 19
    elif (x=='LBN'):
        return 20
    elif (x=='TUR'):
        return 21
    elif (x=='IND'):
        return 22
    
def process_domain(x):
    """
    Process the domain features
    """
    dict_ = {'yahoo.fr':0, 'hotmail.fr':1, 'hotmail.com': 2, 'gmail.com': 3, 'orange.fr': 4, 'outlook.com': 5, 'free.fr': 6, 'laposte.net': 7, 'other': 8, 'neuf.fr': 9, 'wanadoo.fr': 10, 'me.com': 11, 'ymail.com': 12, 'sfr.fr': 13, 'live.fr': 14, 'bbox.fr': 15, 'outlook.fr': 16, 'msn.com': 17, 'yahoo.com': 18, 'aol.com': 19, 'icloud.com': 20, 'cegetel.net': 21, 'club-internet.fr': 22}
    x['domain'] = x['domain'].map(dict_)
    return x

In [292]:
X, y, data  = data_management(df_initial)

In [293]:
data

,campagne,code_postal,code_postal_naissance,csp,debit_cb,flag_banque_principale,flag_epargne,nature_cb,patrimoine,regime_matrimonial,revenus_annuels,cible,AGE,Parrain,domain,Sex,country_connexion,pays_de_naissance
1,456.0,31130.0,38000.0,2.0,0.0,0.0,0.0,1.0,2.0,4.0,2.0,0,47.0,0,0.0,1,1.0,1.0
3,300.0,94800.0,93420.0,2.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,0,28.0,0,1.0,0,1.0,1.0
5,307.0,73150.0,92290.0,2.0,0.0,0.0,0.0,1.0,3.0,2.0,2.0,0,54.0,0,2.0,1,1.0,1.0
7,868.0,69001.0,69008.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1,27.0,1,3.0,0,1.0,1.0
8,868.0,75015.0,42000.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0,28.0,1,3.0,0,1.0,1.0
17,1849.0,72550.0,72000.0,4.0,0.0,0.0,0.0,1.0,1.0,6.0,2.0,0,29.0,0,3.0,-1,1.0,1.0
22,868.0,44130.0,64000.0,15.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1,32.0,0,3.0,1,1.0,1.0
26,868.0,95210.0,95100.0,4.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0,29.0,1,3.0,1,1.0,1.0
38,868.0,35510.0,62000.0,2.0,0.0,0.0,0.0,1.0,3.0,2.0,3.0,0,47.0,0,4.0,0,1.0,1.0
43,868.0,92190.0,92290.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0,31.0,1,0.0,0,1.0,1.0


In [294]:
del data['debit_cb']

In [295]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=54)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7032, 17)
(3015, 17)
(7032,)
(3015,)


In [234]:
X_test

,campagne,code_postal,code_postal_naissance,csp,debit_cb,flag_banque_principale,flag_epargne,nature_cb,patrimoine,regime_matrimonial,revenus_annuels,AGE,Parrain,domain,Sex,country_connexion,pays_de_naissance
21339,300.0,31100.0,31100.0,4.0,1.0,1.0,1.0,0.0,0.0,6.0,2.0,31.0,0,4.0,0,1.0,1.0
22073,300.0,94300.0,54520.0,13.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,35.0,0,1.0,1,1.0,1.0
14799,1176.0,75014.0,78600.0,4.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,49.0,0,3.0,0,1.0,1.0
23548,868.0,13006.0,53100.0,4.0,2.0,0.0,0.0,0.0,0.0,4.0,1.0,27.0,1,3.0,0,1.0,1.0
7126,868.0,70000.0,25200.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,30.0,1,11.0,1,1.0,1.0
7498,456.0,59790.0,59163.0,12.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,18.0,0,20.0,1,1.0,1.0
26423,307.0,33140.0,40000.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,28.0,0,3.0,1,1.0,1.0
9484,868.0,69150.0,69150.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,21.0,1,1.0,0,1.0,1.0
17660,868.0,83640.0,59113.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,40.0,1,1.0,-1,1.0,1.0
8082,868.0,33000.0,92310.0,12.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,28.0,1,3.0,1,1.0,1.0


In [302]:
# Modèle 3: nearest_centroid
from sklearn.neighbors.nearest_centroid import NearestCentroid
n_neighbors = 15
KNC = NearestCentroid()
KNC.fit(X_train, y_train)
y11_pred = KNC.predict(X_test)

print(classification_report(y_test, y11_pred))
print ('accuracy score : '+ str(accuracy_score(y_test, y11_pred)))
print ('\n confussion matrix:\n'+ str(confusion_matrix(y_test, y11_pred)))

             precision    recall  f1-score   support

          0       0.77      0.44      0.56      2293
          1       0.25      0.58      0.35       722

avg / total       0.64      0.47      0.51      3015

accuracy score : 0.473300165837

 confussion matrix:
[[1006 1287]
 [ 301  421]]


In [303]:
### Modèle 1 : RDF
classifier= RandomForestClassifier(n_estimators=200,max_depth=210)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print ('accuracy score : '+ str(accuracy_score(y_test, y_pred)))
print('classification_report:'+ str(classification_report(y_test, y_pred)))
print ('\n confussion matrix:\n'+ str(confusion_matrix(y_test, y_pred)))

accuracy score : 0.763847429519
classification_report:             precision    recall  f1-score   support

          0       0.78      0.97      0.86      2293
          1       0.53      0.11      0.18       722

avg / total       0.72      0.76      0.70      3015


 confussion matrix:
[[2223   70]
 [ 642   80]]


In [304]:
                     ######################## Testing with balanced labels #########################
#Running Easy Ensemble
import imblearn.ensemble as ensemble
EE = ensemble.EasyEnsemble()
print(X_train.shape)
ancien_y = pd.DataFrame(y_train, columns=["cible"])
print("Ancien: ", ancien_y[(ancien_y["cible"] == 1)].shape[0], "positives ", 
      ancien_y[(ancien_y["cible"] == 0)].shape[0], "negatives")
sets_X, sets_y = EE.fit_sample(X_train.as_matrix(), y_train.as_matrix())
balanced_X = np.array(sets_X).reshape(-1, X_train.shape[1])
balanced_y = np.array(sets_y).reshape(-1, 1)
print(balanced_X.shape)
# Create temporary DataFrame
featureNames =["campagne","code_postal", "code_postal_naissance", "csp", "debit_cb", "flag_banque_principale", "flag_epargne", "nature_cb", "patrimoine", "regime_matrimonial", "revenus_annuels", "AGE", "Parrain", "domain", "Sex", "country_connexion", "pays_de_naissance"]
X_train_temp = pd.DataFrame(balanced_X, columns=featureNames)
temp_y = pd.DataFrame(balanced_y, columns=["cible"])
y_train_temp = temp_y["cible"]
print("Original: ", temp_y[(temp_y["cible"] == 1)].shape[0], "positives ", 
      temp_y[(temp_y["cible"] == 0)].shape[0], "negatives")

(7032, 17)
('Ancien: ', 1698, 'positives ', 5334, 'negatives')
(33960, 17)
('Original: ', 16980, 'positives ', 16980, 'negatives')


In [305]:
# Modèle 1 : RDF
forest_b= RandomForestClassifier(n_estimators=200,max_depth=201)
forest_b.fit(X_train_temp, y_train_temp)
y_pred_b = forest_b.predict(X_test)
print ('accuracy score : '+ str(accuracy_score(y_test, y_pred_b)))
print('\n classification_report:\n'+ str(classification_report(y_test, y_pred_b)))
print ('\n confussion matrix:\n'+ str(confusion_matrix(y_test, y_pred_b)))

accuracy score : 0.750912106136

 classification_report:
             precision    recall  f1-score   support

          0       0.78      0.94      0.85      2293
          1       0.45      0.16      0.24       722

avg / total       0.70      0.75      0.70      3015


 confussion matrix:
[[2146  147]
 [ 604  118]]


In [306]:
# Modèle 2: MLP
from sklearn.neural_network import MLPClassifier
clf_mlp = MLPClassifier(hidden_layer_sizes=(13, 24), random_state=50)
clf_mlp.fit(X_train_temp, y_train_temp)
y2_pred = clf_mlp.predict(X_test)

print(classification_report(y_test, y2_pred))
print ('accuracy score : '+ str(accuracy_score(y_test, y2_pred)))
print ('\n confussion matrix:\n'+ str(confusion_matrix(y_test, y2_pred)))

             precision    recall  f1-score   support

          0       0.76      0.94      0.84      2293
          1       0.24      0.06      0.09       722

avg / total       0.64      0.73      0.66      3015

accuracy score : 0.731343283582

 confussion matrix:
[[2165  128]
 [ 682   40]]


In [307]:
# Modèle 3: nearest_centroid
from sklearn.neighbors.nearest_centroid import NearestCentroid
KNC = NearestCentroid()
KNC.fit(X_train_temp, y_train_temp)
y3_pred = KNC.predict(X_test)

print(classification_report(y_test, y3_pred))
print ('accuracy score : '+ str(accuracy_score(y_test, y3_pred)))
print ('\n confussion matrix:\n'+ str(confusion_matrix(y_test, y3_pred)))

             precision    recall  f1-score   support

          0       0.77      0.44      0.56      2293
          1       0.25      0.58      0.35       722

avg / total       0.64      0.47      0.51      3015

accuracy score : 0.473300165837

 confussion matrix:
[[1006 1287]
 [ 301  421]]


# Avec parrain

In [376]:
# df_init_p = pd.read_csv('df_new_initial.csv', sep=',') 
df_init_p = pd.read_csv('result.csv', sep=',') 
del df_init_p ['Unnamed: 0']
del df_init_p ['cible']
df_init_p

,contactid,annee_mois,campagne,campaign,civilite,code_postal,code_postal_naissance,country_connexion_name,csp,date_naissance,...,nature_cb,patrimoine,pays,pays_naissance,regime_matrimonial,revenus_annuels,service,timestamp,id_dim_personne,encours
0,6546762003,NaN,-1.0,Source URL non trouvée,NaN,NaN,59430.0,France,15.0,NaN,...,NaN,-1.0,NaN,FRA,2.0,2.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-09-21T01:28:13.601+0200,6546762003,5.00
1,6742433330,NaN,456.0,REC_Liens textes autopromotion 456,0,31130.0,38000.0,France,2.0,11/04/1969,...,1.0,2.0,FRA,FRA,4.0,2.0,SFOL_AJOUTER_CB,2016-10-31T18:16:38.719+0100,6742433330,0.00
2,6856828837,201611.0,868.0,PAR_Onlin_Site _ECard,MR,92000.0,92150.0,France,2.0,11/05/1957,...,NaN,2.0,FRA,FRA,2.0,4.0,monprofil.PROSPECT/CREER_FICHE_PROSPECT,2016-11-21T12:01:44.736+0100,6856828837,8899.93
3,6754882186,NaN,300.0,Intb_Banque_Autre,1,94800.0,93420.0,France,2.0,24/06/1988,...,1.0,0.0,FRA,FRA,2.0,2.0,SFOL_AJOUTER_CB,2016-11-04T10:46:58.447+0100,6754882186,10.00
4,6289817192,201611.0,300.0,Intb_Banque_Autre,MR,26170.0,84600.0,Switzerland,2.0,01/05/1994,...,NaN,0.0,FRA,FRA,0.0,2.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-11-21T17:36:53.976+0100,6289817192,173.53
5,6748699778,NaN,307.0,REC_Onlin_MC_Googl,0,73150.0,92290.0,France,2.0,28/10/1962,...,1.0,3.0,FRA,FRA,2.0,2.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-11-01T13:44:10.556+0100,6748699778,10.00
6,6856855432,201612.0,307.0,REC_Onlin_MC_Googl,MR,75015.0,75015.0,France,18.0,08/10/1992,...,NaN,0.0,FRA,FRA,0.0,1.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-12-13T19:17:18.159+0100,6856855432,234.08
7,6821231675,201611.0,868.0,PAR_Onlin_Site _ECard,2,69001.0,69008.0,France,3.0,07/08/1989,...,0.0,0.0,FRA,FRA,0.0,1.0,SFOL_AJOUTER_CB,2016-11-15T16:46:02.913+0100,6821231675,3442.00
8,6747566365,NaN,868.0,PAR_Onlin_Site _ECard,2,75015.0,42000.0,France,2.0,09/06/1988,...,1.0,0.0,FRA,FRA,0.0,2.0,SFOL_AJOUTER_CB,2016-11-01T10:11:34.523+0100,6747566365,380.00
9,6888675097,201611.0,1543.0,REC_Onlin_Parte_Bours_SP,MLE,59161.0,59300.0,France,12.0,13/06/1996,...,NaN,0.0,FRA,FRA,0.0,0.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2016-11-26T17:13:18.006+0100,6888675097,319.63


In [377]:
#Fixer encours des bons clients 
df_init_p['cible'] = df_init_p['encours'].map(lambda x: 0 if x < 1100.00 else 1).astype(int)
df_init_p[['encours','cible']].head()

,encours,cible
0,5.00,0
1,0.00,0
2,8899.93,1
3,10.00,0
4,173.53,0


In [378]:
### Data management
def data_management_p(df):
    
    ### Fill empty value
    
    df['code_postal'] = df['code_postal'].replace('',-2)
    df['code_postal'] = pd.to_numeric(df['code_postal'])
    
    df['code_postal_naissance'] = pd.to_numeric(df['code_postal_naissance'])
   
    df['country_connexion_name'] = df['country_connexion_name'].replace(['',None],-2)
    
    df['date_naissance'] = df['date_naissance'].replace('','01/01/1800')
    
    df['flag_epargne'] = pd.to_numeric(df['flag_epargne'])
    
    #df['mailing_accord'] = df['mailing_accord'].replace(['',None],-2)
    #df['mailing_accord'] = pd.to_numeric(df['mailing_accord'])
    del df['mailing_accord']
    
    df['nature_cb'] = df['nature_cb'].replace(['',None],-2)
    df['nature_cb'] = pd.to_numeric(df['nature_cb'])
    
    df['pays_naissance'] = df['pays_naissance'].replace(['',None],-2)
    
    ### get age
    get_age(df)
    

    ### Get flag parrain from namesponsor
    #df['Parrain'] = df['namesponsor'].map(lambda x: 0 if pd.isnull(x) else 1).astype(int)
    del df['firstnamesponsor'] 
    del df['namesponsor'] 
    
    
    ### Get domain from mail
    df['mail'] = df.mail.replace ([np.nan], '')
    df['domain'] = df['mail'].map(lambda x: x.split("@")[1] if x else None)
    df['domain'] = df.domain.str.lower()
    del df['mail']

    ### Get Sex from Civility
    df['Sex'] = df['civilite'].map(get_sex)
    del df['civilite']
    
    ### Delete useless cols
    del df['annee_mois']
    del df['id_dim_temps']
    del df['id_dim_personne']
    del df['encours']
    del df['timestamp']
    del df['service']
    del df['campaign']

    del df['pays']
    
    ### transform to numeric when possible
    df = df.apply(lambda x: pd.to_numeric(x, errors='ignore'))
    df['country_connexion_name'] = df['country_connexion_name'].replace ([np.nan], '')
    df['domain'] = df['domain'].replace ([np.nan], '')
    df['pays_naissance'] = df['pays_naissance'].replace ([np.nan], '')
    df.loc[df['country_connexion_name'].value_counts()[df['country_connexion_name']].values < 40, 'country_connexion_name'] = 'other'
    df.loc[df['domain'].value_counts()[df['domain']].values < 40, 'domain'] = 'other'
    df.loc[df['pays_naissance'].value_counts()[df['pays_naissance']].values < 40, 'pays_naissance'] = 'other'
    
    
    ### Get country connexion name
    df['country_connexion'] = df['country_connexion_name'].map(process_country_connexion)
    del df['country_connexion_name']
    ### Get pays de naissance
    df['pays_de_naissance'] = df['pays_naissance'].map(process_pays_naissance)
    del df['pays_naissance']
    
    ### Process the domain
    df = process_domain (df)
    
    ### Drop NaN 
    df = df.dropna()
    
    return df


def get_sex(x):
    if x == "0":
        return 1
    elif (x == "1"):
        return 0
    elif (x == "2"):
        return 0
    elif (x == "MR"):
        return 1
    elif (x == "MLE"):
        return 0
    elif (x == "MME"):
        return 0
    else :
        return -1

def get_age(X):
    from datetime import datetime
    now = datetime.now()
    X['AGE'] = X['date_naissance'].dropna()
    X['AGE'] = pd.to_datetime(X['AGE'],errors='coerce')
    X['AGE'] = X['AGE'].map(lambda x : now.year-x.year  if now.month-x.month>0 else now.year - x.year -1 )
    del X['date_naissance']
    
def process_country_connexion(x):
    
    if x == 'France':
        return 1
    elif (x == 'United Kingdom'):
        return 2
    elif (x == 'other'):
        return 3
    elif (x == 'Germany'):
        return 4
    elif (x == 'Switzerland'):
        return 5
    elif (x == 'Netherlands'):
        return 6
    elif (x == 'Europe'):
        return 7
    elif (x == 'United States'):
        return 8
    elif (x == 'Reunion'):
        return 9
    elif (x==-2):
        return -2
    
def process_pays_naissance(x):

    if x == 'FRA':
        return 1
    elif (x == 'ITA'):
        return 2
    elif (x == 'other'):
        return 3
    elif (x == 'DZA'):
        return 4
    elif (x == 'BEN'):
        return 5
    elif (x == 'ESP'):
        return 6
    elif (x == 'VNM'):
        return 7
    elif (x == 'DEU'):
        return 8
    elif (x == 'MAR'):
        return 9
    elif (x=='CIV'):
        return 10
    elif (x=='CMR'):
        return 10
    elif (x=='GTO'):
        return 11
    elif (x=='SEN'):
        return 12
    elif (x=='BEL'):
        return 13
    elif (x=='CHN'):
        return 14
    elif (x=='ROU'):
        return 15
    elif (x=='BRA'):
        return 16
    elif (x=='MDG'):
        return 17
    elif (x=='PRT'):
        return 18
    elif (x=='GBR'):
        return 19
    elif (x=='LBN'):
        return 20
    elif (x=='TUR'):
        return 21
    elif (x=='IND'):
        return 22
    
def process_domain(x):
    """
    Process the domain features
    """
    dict_ = {'yahoo.fr':0, 'hotmail.fr':1, 'hotmail.com': 2, 'gmail.com': 3, 'orange.fr': 4, 'outlook.com': 5, 'free.fr': 6, 'laposte.net': 7, 'other': 8, 'neuf.fr': 9, 'wanadoo.fr': 10, 'me.com': 11, 'ymail.com': 12, 'sfr.fr': 13, 'live.fr': 14, 'bbox.fr': 15, 'outlook.fr': 16, 'msn.com': 17, 'yahoo.com': 18, 'aol.com': 19, 'icloud.com': 20, 'cegetel.net': 21, 'club-internet.fr': 22}
    x['domain'] = x['domain'].map(dict_)
    return x

In [379]:
data_p = data_management_p(df_init_p)

In [380]:
data_p

,contactid,campagne,code_postal,code_postal_naissance,csp,debit_cb,flag_banque_principale,flag_epargne,nature_cb,patrimoine,regime_matrimonial,revenus_annuels,cible,AGE,domain,Sex,country_connexion,pays_de_naissance
1,6742433330,456.0,31130.0,38000.0,2.0,0.0,0.0,0.0,1.0,2.0,4.0,2.0,0,47.0,0.0,1,1.0,1.0
3,6754882186,300.0,94800.0,93420.0,2.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,0,28.0,1.0,0,1.0,1.0
5,6748699778,307.0,73150.0,92290.0,2.0,0.0,0.0,0.0,1.0,3.0,2.0,2.0,0,54.0,2.0,1,1.0,1.0
7,6821231675,868.0,69001.0,69008.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1,27.0,3.0,0,1.0,1.0
8,6747566365,868.0,75015.0,42000.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0,28.0,3.0,0,1.0,1.0
17,6701524556,1849.0,72550.0,72000.0,4.0,0.0,0.0,0.0,1.0,1.0,6.0,2.0,0,29.0,3.0,-1,1.0,1.0
22,6738608856,868.0,44130.0,64000.0,15.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1,32.0,3.0,1,1.0,1.0
26,5648776005,868.0,95210.0,95100.0,4.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0,29.0,3.0,1,1.0,1.0
38,6805307194,868.0,35510.0,62000.0,2.0,0.0,0.0,0.0,1.0,3.0,2.0,3.0,0,47.0,4.0,0,1.0,1.0
43,6727392192,868.0,92190.0,92290.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0,31.0,0.0,0,1.0,1.0


In [381]:
Parrain_df = pd.read_csv('parrain_totale_afteradd14.csv', sep=',')
del Parrain_df['Unnamed: 0']
Parrain_df

,id_dim_personne,encours_parrain,id_parrain
0,1004250808,2475.93,2.241080e+08
1,1660202716,NaN,NaN
2,1715100837,2736.14,1.697959e+09
3,1725735525,NaN,NaN
4,1748840201,1311.33,8.146321e+08
5,1766903833,1498.94,2.797564e+08
6,1887361163,NaN,NaN
7,1989548372,16516.12,1.963780e+09
8,2232679265,NaN,NaN
9,2260529045,NaN,NaN


In [382]:
# Merge parrain id with DF initial selon l'id client
enriched_df = pd.merge(data_p, Parrain_df, right_on='id_dim_personne', left_on='contactid')
enriched_df

,contactid,campagne,code_postal,code_postal_naissance,csp,debit_cb,flag_banque_principale,flag_epargne,nature_cb,patrimoine,...,revenus_annuels,cible,AGE,domain,Sex,country_connexion,pays_de_naissance,id_dim_personne,encours_parrain,id_parrain
0,6742433330,456.0,31130.0,38000.0,2.0,0.0,0.0,0.0,1.0,2.0,...,2.0,0,47.0,0.0,1,1.0,1.0,6742433330,19144.72,5.516969e+09
1,6754882186,300.0,94800.0,93420.0,2.0,0.0,0.0,0.0,1.0,0.0,...,2.0,0,28.0,1.0,0,1.0,1.0,6754882186,NaN,NaN
2,6748699778,307.0,73150.0,92290.0,2.0,0.0,0.0,0.0,1.0,3.0,...,2.0,0,54.0,2.0,1,1.0,1.0,6748699778,NaN,NaN
3,6821231675,868.0,69001.0,69008.0,3.0,2.0,1.0,0.0,0.0,0.0,...,1.0,1,27.0,3.0,0,1.0,1.0,6821231675,1028.36,2.306432e+09
4,6747566365,868.0,75015.0,42000.0,2.0,0.0,0.0,1.0,1.0,0.0,...,2.0,0,28.0,3.0,0,1.0,1.0,6747566365,587.07,4.942456e+09
5,6701524556,1849.0,72550.0,72000.0,4.0,0.0,0.0,0.0,1.0,1.0,...,2.0,0,29.0,3.0,-1,1.0,1.0,6701524556,NaN,NaN
6,6738608856,868.0,44130.0,64000.0,15.0,2.0,1.0,1.0,0.0,0.0,...,1.0,1,32.0,3.0,1,1.0,1.0,6738608856,0.00,9.814904e+08
7,5648776005,868.0,95210.0,95100.0,4.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0,29.0,3.0,1,1.0,1.0,5648776005,139.93,5.913001e+09
8,6805307194,868.0,35510.0,62000.0,2.0,0.0,0.0,0.0,1.0,3.0,...,3.0,0,47.0,4.0,0,1.0,1.0,6805307194,NaN,NaN
9,6727392192,868.0,92190.0,92290.0,2.0,0.0,0.0,0.0,1.0,1.0,...,2.0,0,31.0,0.0,0,1.0,1.0,6727392192,18112.32,3.130519e+09


In [385]:
### Data management parrain
def data_management_parrain(df):
    
    ### Get flag parrain from id parrain
    df['Parrain'] = df['id_parrain'].map(lambda x: 0 if pd.isnull(x) else 1).astype(int)
    del df['id_parrain']
    

    ### Get flag encours_parrain (1000) from encours_parrain 
    df['encours_p'] = df['encours_parrain'].map(lambda x : 0 if x<1000 or pd.isnull(x) else 1).astype(int)
    del df['encours_parrain']
    
    ### USELESS COLUMNS
    
    del df['contactid']
    del df['id_dim_personne'] 
    
    features_df = df.drop('cible', axis=1)
    target = df['cible']
    
    return features_df, target



In [386]:
X_parrain, y_parrain = data_management_parrain(enriched_df)

In [387]:
X_parrain.head()

,campagne,code_postal,code_postal_naissance,csp,debit_cb,flag_banque_principale,flag_epargne,nature_cb,patrimoine,regime_matrimonial,revenus_annuels,AGE,domain,Sex,country_connexion,pays_de_naissance,encours_p,Parrain
0,456.0,31130.0,38000.0,2.0,0.0,0.0,0.0,1.0,2.0,4.0,2.0,47.0,0.0,1,1.0,1.0,1,1
1,300.0,94800.0,93420.0,2.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,28.0,1.0,0,1.0,1.0,0,0
2,307.0,73150.0,92290.0,2.0,0.0,0.0,0.0,1.0,3.0,2.0,2.0,54.0,2.0,1,1.0,1.0,0,0
3,868.0,69001.0,69008.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,27.0,3.0,0,1.0,1.0,1,1
4,868.0,75015.0,42000.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,28.0,3.0,0,1.0,1.0,0,1


In [388]:
X02_train, X02_test, y02_train, y02_test = train_test_split (X_parrain,y_parrain,test_size=0.30,random_state=42)

In [389]:
# MLP
from sklearn.neural_network import MLPClassifier
clf2_mlp = MLPClassifier(hidden_layer_sizes=(11, 12), random_state=50)
clf2_mlp.fit(X02_train, y02_train)
yp_pred = clf2_mlp.predict(X02_test)

print(classification_report(y02_test, yp_pred))
print ('accuracy score : '+ str(accuracy_score(y02_test, yp_pred)))
print ('\n confussion matrix:\n'+ str(confusion_matrix(y02_test, yp_pred)))

             precision    recall  f1-score   support

          0       0.75      0.31      0.44      2237
          1       0.26      0.71      0.38       778

avg / total       0.63      0.41      0.43      3015

accuracy score : 0.414262023217

 confussion matrix:
[[ 699 1538]
 [ 228  550]]


In [390]:
# Modèle 3: nearest_centroid
from sklearn.neighbors.nearest_centroid import NearestCentroid
KNC2 = NearestCentroid()
KNC2.fit(X02_train, y02_train)
y31_pred = KNC2.predict(X02_test)

print(classification_report(y02_test, y31_pred))
print ('accuracy score : '+ str(accuracy_score(y02_test, y31_pred)))
print ('\n confussion matrix:\n'+ str(confusion_matrix(y02_test, y31_pred)))

             precision    recall  f1-score   support

          0       0.73      0.36      0.48      2237
          1       0.25      0.63      0.36       778

avg / total       0.61      0.43      0.45      3015

accuracy score : 0.426533996683

 confussion matrix:
[[ 796 1441]
 [ 288  490]]


In [391]:
# Modèle 1 : RDF
forest_b= RandomForestClassifier(n_estimators=200,max_depth=201)
forest_b.fit(X_train_temp, y_train_temp)
y_pred_b = forest_b.predict(X_test)
print ('accuracy score : '+ str(accuracy_score(y_test, y_pred_b)))
print('\n classification_report:\n'+ str(classification_report(y_test, y_pred_b)))
print ('\n confussion matrix:\n'+ str(confusion_matrix(y_test, y_pred_b)))

accuracy score : 0.749585406302

 classification_report:
             precision    recall  f1-score   support

          0       0.78      0.94      0.85      2293
          1       0.43      0.15      0.23       722

avg / total       0.70      0.75      0.70      3015


 confussion matrix:
[[2150  143]
 [ 612  110]]


# Proba

In [ ]:
u= cible
v="proba"
probabilite = "%s_%s" %(u,v)
predicted_target = "predicted_target_" + cible